# Fine tuning

In [1]:
config = {
    "lr" : 1e-6,
    "epochs" : 5    
    }

## Set up

In [2]:
!nvidia-smi # to see what GPU you have

/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: _moduleraw: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `_moduleraw'
Wed Apr 26 01:28:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   22C    P8    13W /  70W |      2MiB / 15360MiB |      0%    

In [3]:
# !pip install wandb --quiet
# !pip install torchsummaryX -q
# !pip install mutagen
# !pip install jiwer
# !pip install git+https://github.com/openai/whisper.git 
# # on Ubuntu or Debian
# !sudo apt update && sudo apt install ffmpeg
# !pip install -qqq evaluate==0.2.2

In [3]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['CUDA_PATH']= '/usr/local/cuda-11.7'

In [4]:
# print(os.environ['CUDA_PATH'])

In [4]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from tqdm import tqdm

import torchaudio.transforms as tat

# from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

from mutagen.mp3 import MP3
import jiwer

import wandb
import evaluate

import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

if DEVICE == 'cuda':
    dtype = torch.float16
else:
    dtype = torch.float32

Device:  cuda


## Load the pretrained Whisper model and tokenizer

In [5]:
import whisper

model = whisper.load_model("tiny")
tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language="en", task="transcribe")

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 111MiB/s]


In [6]:
# hard-coded audio hyperparameters
SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 80
HOP_LENGTH = 160
CHUNK_LENGTH = 30
N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000 samples in a 30-second chunk
N_FRAMES = whisper.utils.exact_div(N_SAMPLES, HOP_LENGTH)  # 3000 frames in a mel spectrogram input

N_SAMPLES_PER_TOKEN = HOP_LENGTH * 2  # the initial convolutions has stride 2
FRAMES_PER_SECOND = whisper.utils.exact_div(SAMPLE_RATE, HOP_LENGTH)  # 10ms per audio frame
TOKENS_PER_SECOND = whisper.utils.exact_div(SAMPLE_RATE, N_SAMPLES_PER_TOKEN)  # 20ms per audio token

In [7]:
torch.cuda.empty_cache()
gc.collect()

517

## Dataloader

In [29]:
class AudioDataloader(torch.utils.data.DataLoader):

    # For this homework, we give you full flexibility to design your data set class.
    
    def __init__(self, partition, root, tokenizer, batch_size, shuffle): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''
        self.df = pd.read_csv('common_voice_'+partition+'.csv')
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.df_len = len(self.df)
        self.root = root
        self.shuffle = shuffle
        

    def __len__(self):
        
        return round((self.df_len//self.batch_size)*0.5)
    
    def __iter__(self):
        if self.shuffle:
            self.df = self.df.sample(frac = 1)
        num_batches = self.__len__()
        batch_idx = 0
        batch_mfccs = []
        batch_transcripts = []
        batch_probs = []
        for index, entry in self.df.iterrows():
            filepath = self.root + entry['path']
            try:
                audio = MP3(filepath)
                if audio.info.length < 30:
                    transcript = tokenizer.encode(entry['sentence'])
                    prob = entry['probs']
                    mfcc = whisper.audio.log_mel_spectrogram(filepath,padding=N_SAMPLES)
                    mfcc = whisper.audio.pad_or_trim(mfcc, N_FRAMES).to(DEVICE).to(dtype)
                    batch_mfccs.append(mfcc)
                    batch_transcripts.append(transcript)
                    batch_probs.append(prob)
                if len(batch_probs)== self.batch_size:
                    batch_idx += 1
                    yield self.collate_fn((batch_mfccs,batch_transcripts,torch.LongTensor(batch_probs)))
                    batch_mfccs = []
                    batch_transcripts = []
                    batch_probs = []
            except:
                pass
        self.length = len(self.mfccs)

    def collate_fn(self,batch):
        # batch of input mfcc coefficients
        batch_mfcc = [] # TODO
        # batch of output phonemes
        batch_transcript = [] # TODO
        # batch_content_frame = []
        batch_prob = []
        lengths_mfcc = []
        lengths_transcript = []

        # for mfcc,transcript,content_frame, prob in batch:
        for mfcc,transcript, prob in batch:
        
          batch_mfcc.append(mfcc)
          batch_transcript.append(transcript)
          # batch_content_frame.append(content_frame)
          batch_prob.append(prob)
          lengths_mfcc.append(len(mfcc))
          lengths_transcript.append(len(transcript))
          
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first= True) # TODO
        batch_transcript_pad = pad_sequence(batch_transcript, batch_first= True) # TODO
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(batch_prob), torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)


In [30]:
BATCH_SIZE = 10 # Increase if your device can handle it
num_updates = 200
val_updates = 200

In [31]:
# Create objects for the dataset class
root = '/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/'
# train_data = AudioDataset('train',root, tokenizer) #TODO
# val_data = AudioDataset('validation',root, tokenizer) # TODO : You can either use the same class with some modifications or make a new one :)

# # Do NOT forget to pass in the collate function as parameter while creating the dataloader
# train_loader = torch.utils.data.DataLoader(
#     dataset     = train_data, 
#     num_workers = 0,
#     batch_size  = BATCH_SIZE, 
# #     pin_memory  = True,
#     shuffle     = True,
#     collate_fn  = train_data.collate_fn
# )
# val_loader = torch.utils.data.DataLoader(
#     dataset     = val_data,
#     num_workers = 0,
#     batch_size  = BATCH_SIZE, 
#     # pin_memory  = True,
#     shuffle     = False,
#     collate_fn  = val_data.collate_fn
# )

# print("Batch size: ", BATCH_SIZE)
# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
# print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
train_loader = AudioDataloader('train', root, tokenizer, BATCH_SIZE, shuffle = True)
val_loader = AudioDataloader('validation', root, tokenizer, BATCH_SIZE, shuffle = True)

In [ ]:
# sanity check
x, y, p, lx, ly = next(iter(train_loader))
print(x.shape, y.shape, p.shape, lx.shape, ly.shape)
print(x,y,p,lx,ly)
x, y, p, lx, ly = next(iter(train_loader))
print(x.shape, y.shape, p.shape, lx.shape, ly.shape)
print(x,y,p,lx,ly)

## Training Model

In [10]:
class ASRModel(torch.nn.Module):

    def __init__(self, model):
        super().__init__()
        self.options = whisper.decoding.DecodingOptions(fp16 = True,temperature=0.0,without_timestamps=True)
        self.asr = model
        
    
    def forward(self, x, y):
        # print("x type",type(x))
        # print("x shape",x.shape)
        # print("opt type",type(self.options))
        # print("opt shape",self.options.shape)
        # print("length",lengths_x)
        audio_features = self.asr.encoder(x)
        results = self.asr.decoder(y, audio_features)
        # print(results.shape,results)
        results = results.log_softmax(2)
        # print(results.shape,results)
        # out = [torch.LongTensor(result.tokens).to(DEVICE) for result in results]
        # out = torch.nn.utils.rnn.pad_sequence(out, batch_first=True, padding_value=0.0)
        lx = self.asr.decode(x,self.options)
        # print("orig len", len(out))
        lx = list(map(lambda x: len(x.tokens), lx))
        # seq_unpacked, lens_unpacked = pad_packed_sequence(out, batch_first = True)
        # print("new len",len(out))
        lx = torch.LongTensor(lx).to(DEVICE)
        # return out, lengths_x
        # return seq_unpacked, lens_unpacked
        # print(type(out))
        # print(out.shape)
        # out = out.double()
        # return out, lengths_x
        return results,lx

## Criterion, optimizer, scheduler

In [11]:
# criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True) # Define CTC loss as the criterion. How would the losses be reduced?
criterion = torch.nn.CrossEntropyLoss(reduction = 'none')
optimizer =  torch.optim.SGD(model.parameters(),lr = config["lr"]) # What goes in here?

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2) #TODO
# metric = evaluate.load("wil")
# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Train and validate functions

In [20]:

# def train_model(model, train_loader, optimizer):
#     model.train()
#     batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

#     total_loss = 0

#     for i, data in enumerate(train_loader):
#         optimizer.zero_grad()

#         x, y, p, lx, ly = data
#         x, y, p = x.to(DEVICE), y.to(DEVICE),p.to(DEVICE)

#         with torch.cuda.amp.autocast():     
#             h,lh = model(x, y)
#             h = torch.permute(h, (1, 0, 2))
#             loss = criterion(h, y, ly, ly)
            
#         total_loss += loss.item()

#         batch_bar.set_postfix(
#             loss="{:.04f}".format(float(total_loss / (i + 1))),
#             lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

#         batch_bar.update() # Update tqdm bar

#         # Another couple things you need for FP16. 
#         scaler.scale(loss).backward() # This is a replacement for loss.backward()
#         scaler.step(optimizer) # This is a replacement for optimizer.step()
#         scaler.update() # This is something added just for FP16

#         del x, y, lx, ly, h, lh, loss 
#         torch.cuda.empty_cache()

#     batch_bar.close() # You need this to close the tqdm bar
    
#     return total_loss / len(train_loader)


# def validate_model(model, val_loader):

#     model.eval()
#     batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

#     total_loss = 0
#     val_wil = 0

#     for i, data in enumerate(val_loader):

#         x, y, p, lx, ly = data
#         x, y = x.to(DEVICE), y.to(DEVICE)

#         with torch.inference_mode():
#             # # h, lh = model(x, lx)
#             # h = model(x, y)
#             # h = torch.permute(h,(0,2,1))
#             # # h = torch.permute(h, (1, 0, 2))
#             # # loss = criterion(h, y, lh, ly)
#             # # print(y.shape)
#             # # print(y)
#             # loss = criterion(h,y)
#             h,lh = model(x, y)
#             h = torch.permute(h, (1, 0, 2))
#             loss = criterion(h, y, ly, ly)
#             ht = tokenizer.decode(h)
#             hy 
#             # loss = torch.mean(loss)

#         total_loss += float(loss)
#         # val_wil += wil(h,y)

#         batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(val_wil / (i + 1))))

#         batch_bar.update()
    
#         del x, y, lx, ly, h, lh, loss
#         torch.cuda.empty_cache()
        
#     batch_bar.close()
#     total_loss = total_loss/len(val_loader)
#     avg_wil = val_wil/len(val_loader)
#     return total_loss, avg_wil

In [12]:
def validate_model(model, val_loader, val_batches):

    model.eval()
    batch_bar = tqdm(total=val_batches, dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    val_wil = 0

    for i, data in enumerate(val_loader):
        if i == val_batches:
            break
        x, y, p, lx, ly = data
        x, y = x.to(DEVICE), y.to(DEVICE)

        with torch.inference_mode():
            h,lh = model(x, y)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y)
            loss /= p
            loss = sum(loss)
            # loss = torch.mean(loss)
        total_loss += float(loss)
        # val_wil += wil(h,y)
        
        ht = model.asr.decode(x,model.options)
        ht = list(map(lambda x: x.tokens, ht))
        text_x,text_y = [],[]
        for j in range(len(ht)):
            val_wil += jiwer.wil(tokenizer.decode(ht[j]), tokenizer.decode(y[j]))/len(ht)            

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), wil="{:.04f}".format(float(val_wil / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    avg_wil = val_wil/len(val_loader)
    return total_loss, avg_wil

## Training setup

In [13]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [14]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_wil = 1 # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/data/home/ubuntu/11785-ASR-Project/finetune/epoch_model.checkpoint"#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/data/home/ubuntu/11785-ASR-Project/finetune/best_model.checkpoint"#TODO set best model path 

## wandb

In [15]:
wandb.login(key="a5b7420abbe354e6d0b2f5554b97ee11f327fc92") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: shilinm (deeper_learners). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [16]:
# Create your wandb run
run = wandb.init(
    name = "test-run", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = 'oigknwdr', #Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "whisper-finetune", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

In [17]:
# run.finish()

## Train

In [17]:
asrmodel = ASRModel(model)

In [18]:
val_loss, val_wil = validate_model(asrmodel,val_loader)

In [19]:
print("start val loss", val_loss, "start val wil", val_wil)

start val loss 14.629981064796448 start val wil 0.5157564043184116


In [20]:
torch.cuda.empty_cache()
gc.collect()

1697

In [21]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    total_loss = 0
    batch_bar = tqdm(total=num_updates, dynamic_ncols=True, leave=True, position=0, desc='Train')
    model.train()
    for i, data in enumerate(train_loader):
        if (i+1)%num_updates==0:
            batch_bar.close()
            train_loss = total_loss / len(train_loader)
            val_loss, val_wil = validate_model(asrmodel,val_loader)
            curr_lr = float(optimizer.param_groups[0]['lr'])
            print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
            print("\tVal Wil {:.04f}\t Val Loss {:.04f}".format(val_wil, val_loss))
            wandb.log({
                'train_loss': train_loss,  
                'val_wil': val_wil, 
                'val_loss': val_loss, 
                'lr'        : curr_lr,
                'epoch'     : i//num_updates,
            })
            save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], i//num_updates, epoch_model_path)
            if val_wil <= best_wil:
                best_wil = val_wil
                save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], i//num_updates, best_model_path)
            scheduler.step(val_loss)
            total_loss = 0
            batch_bar = tqdm(total=num_updates, dynamic_ncols=True, leave=True, position=0, desc='Train')
             
        optimizer.zero_grad()

        x, y, p, lx, ly = data
        x, y, p = x.to(DEVICE), y.to(DEVICE),p.to(DEVICE)

        with torch.cuda.amp.autocast():     
            h,lh = asrmodel(x, y)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y)
            loss /= p
            loss = sum(loss)
            
        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    val_loss, val_wil = validate_model(asrmodel,val_loader)
    save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], epoch, epoch_model_path)
    if val_wil <= best_wil:
        best_wil = val_wil
        save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], epoch, best_model_path)
    # curr_lr = float(optimizer.param_groups[0]['lr'])

    # train_loss              = train_model(asrmodel, train_loader, optimizer)
    # valid_loss, val_wil  = validate_model(asrmodel, val_loader)
    # scheduler.step(valid_loss)
    # step_train_eval_model(asrmodel, train_loader, optimizer, 3, scheduler,val_loader)
    # print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    # print("\tVal Dist {:.04f}\t Val Loss {:.04f}".format(val_wil, valid_loss))



    
    # save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], epoch, epoch_model_path)
#     wandb.save('epoch_model.checkpoint')
    # print("Saved epoch model")

    # if val_wil <= best_wil:
        # best_wil = val_wil
        # save_model(asrmodel, optimizer, scheduler, ['val_wil', val_wil], epoch, best_model_path)
#         wandb.save('best_model.checkpoint')
        # print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
# run.finish()


Epoch: 1/5


Train: 100%|█████████████████████████████▊| 199/200 [01:57<00:00,  1.69it/s, loss=6.1564, lr=0.000001]


	Train Loss 1.2251	 Learning Rate 0.0000010
	Val Wil 0.5230	 Val Loss 4.1584


Train: 100%|██████████████████████████████| 200/200 [02:05<00:00,  1.59it/s, loss=1.7096, lr=0.000001]


	Train Loss 0.6821	 Learning Rate 0.0000010
	Val Wil 0.5232	 Val Loss 2.7573


...

Train: 100%|██████████████████████████████| 200/200 [02:14<00:00,  1.48it/s, loss=0.0683, lr=0.000001]


	Train Loss 0.0546	 Learning Rate 0.0000010
	Val Wil 0.5313	 Val Loss 0.2718


Train: 100%|██████████████████████████████| 200/200 [02:11<00:00,  1.52it/s, loss=0.0525, lr=0.000001]


	Train Loss 0.0524	 Learning Rate 0.0000010
	Val Wil 0.5321	 Val Loss 0.2611


Train:   0%|▏                               | 1/200 [00:00<01:42,  1.95it/s, loss=0.0002, lr=0.000001]


# Playing with the model

In [25]:
# tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language="en", task="transcribe")
# t = tokenizer.encode("Please call Stella. Ask her to bring these things with her from the store: Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids. She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.")
# print(len(t),t)

# # import torchaudio
# # def load_wave(wave_path, sample_rate:int=16000) -> torch.Tensor:
# #     waveform, sr = torchaudio.load(wave_path, normalize=True)
# #     if sample_rate != sr:
# #         waveform = tat.Resample(sr, sample_rate)(waveform)
# #     return waveform
# # audio = load_wave('/content/data/recordings/recordings/english1.mp3', sample_rate=16000)
# # audio = whisper.pad_or_trim(audio.flatten())
# # mel = whisper.log_mel_spectrogram(audio)
# mel = whisper.audio.log_mel_spectrogram('/data/home/ubuntu/cv-corpus-13.0-2023-03-09/en/clips/common_voice_en_22930344.mp3',padding=N_SAMPLES)
# mel = whisper.audio.pad_or_trim(mel, N_FRAMES).to('cuda').to(torch.float32)
# options = whisper.decoding.DecodingOptions(fp16 = False,temperature=0.0)
# m = model.decode(mel,options)
# print(len(m.tokens),m.tokens)
# # d = predict(model, mel)
# # print(d)

79 [16216, 818, 45073, 13, 12320, 720, 281, 1565, 613, 721, 365, 720, 490, 264, 3531, 25, 11678, 36316, 295, 4451, 5756, 24494, 11, 1732, 5060, 1061, 17243, 295, 3344, 5399, 11, 293, 1310, 257, 13288, 337, 720, 3708, 6085, 13, 220, 492, 611, 643, 257, 1359, 5900, 12650, 293, 257, 955, 12058, 17259, 337, 264, 2301, 13, 1240, 393, 19555, 613, 721, 666, 1045, 2182, 10405, 11, 293, 321, 486, 352, 1677, 720, 10579, 412, 264, 3847, 5214, 13]
11 [50364, 18951, 2405, 307, 1071, 1021, 39230, 294, 43120, 13, 50564]


In [11]:
tokenizer.decode([50364, 2555, 818, 45073, 13, 12320, 720, 281, 1565, 613, 721, 365, 720, 490, 264, 3531, 13, 11678, 50608, 50608, 36316, 295, 4451, 5756, 24494, 11, 1732, 5060, 1061, 17243, 295, 3344, 5399, 11, 293, 1310, 257, 13288, 337, 720, 50880, 50880, 3708, 6085, 13, 492, 611, 643, 257, 1359, 5900, 12650, 293, 257, 955, 12058, 17259, 337, 264, 2301, 13, 51120, 51120, 1240, 393, 19555, 613, 721, 666, 1045, 2182, 10405, 293, 321, 486, 352, 1677, 720, 10579, 51390, 51390, 412, 264, 3847, 5214, 13, 51440])

' Please call Stella. Ask her to bring these things with her from the store. Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob. We also need a small plastic snake and a big toy frog for the kids. She can scoop these things into three red bags and we will go meet her Wednesday at the train station.'

In [12]:
tokenizer.decode([16216, 818, 45073, 13, 220, 12320, 720, 281, 1565, 613, 721, 365, 720, 490, 264, 3531, 25, 220, 11678, 36316, 295, 4451, 5756, 24494, 11, 1732, 5060, 1061, 17243, 295, 3344, 5399, 11, 293, 1310, 257, 13288, 337, 720, 3708, 6085, 13, 220, 492, 611, 643, 257, 1359, 5900, 12650, 293, 257, 955, 12058, 17259, 337, 264, 2301, 13, 220, 1240, 393, 19555, 613, 721, 666, 1045, 2182, 10405, 11, 293, 321, 486, 352, 1677, 720, 10579, 412, 264, 3847, 5214, 13])

'Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.'

In [ ]:
tokenizer = whisper.tokenizer.get_tokenizer(model.is_multilingual, language="en", task="transcribe")

In [ ]:
def predict(model, tokenizer, mel):
  decode_options = dict()
  decode_options["fp16"] = (DEVICE == 'cuda')
  if decode_options["fp16"]:
    dtype = torch.float16
  else:
    dtype = torch.float32
  logprob_threshold = -1.0
  no_speech_threshold = 0.6
  decode_options["language"] = "en"


  content_frames = mel.shape[-1] - N_FRAMES

  def decode_with_fallback(segment: torch.Tensor) -> whisper.decoding.DecodingResult:
      kwargs = {**decode_options}
      options = whisper.decoding.DecodingOptions(**kwargs, temperature=0.0)
      decode_result = model.decode(segment, options)
      return decode_result

  seek = 0
  input_stride = whisper.utils.exact_div(
      N_FRAMES, model.dims.n_audio_ctx
  )  # mel frames per output token: 2
  time_precision = (
      input_stride * HOP_LENGTH / SAMPLE_RATE
  )  # time per output token: 0.02 (seconds)
  all_tokens = []
  all_segments = []
  prompt_reset_since = 0

  initial_prompt_tokens = []

  def new_segment(
      *, start: float, end: float, tokens: torch.Tensor, result: whisper.decoding.DecodingResult
  ):
      tokens = tokens.tolist()
      text_tokens = [token for token in tokens if token < tokenizer.eot]
      return {
          "seek": seek,
          "start": start,
          "end": end,
          "text": tokenizer.decode(text_tokens),
          "tokens": tokens,
          "temperature": result.temperature,
          "avg_logprob": result.avg_logprob,
          "compression_ratio": result.compression_ratio,
          "no_speech_prob": result.no_speech_prob,
      }

# show the progress bar when verbose is False (if True, transcribed text will be printed)

  while seek < content_frames:
      time_offset = float(seek * HOP_LENGTH / SAMPLE_RATE)
      mel_segment = mel[:, seek : seek + N_FRAMES]
      segment_size = min(N_FRAMES, content_frames - seek)
      segment_duration = segment_size * HOP_LENGTH / SAMPLE_RATE
      mel_segment = whisper.audio.pad_or_trim(mel_segment, N_FRAMES).to(model.device).to(dtype)

      decode_options["prompt"] = all_tokens[prompt_reset_since:]
      result: whisper.decoding.DecodingResult = decode_with_fallback(mel_segment)
      tokens = torch.tensor(result.tokens)

      if no_speech_threshold is not None:
          # no voice activity check
          should_skip = result.no_speech_prob > no_speech_threshold
          if (
              logprob_threshold is not None
              and result.avg_logprob > logprob_threshold
          ):
              # don't skip if the logprob is high enough, despite the no_speech_prob
              should_skip = False

          if should_skip:
              seek += segment_size  # fast-forward to the next segment boundary
              continue

      previous_seek = seek
      current_segments = []

      timestamp_tokens: torch.Tensor = tokens.ge(tokenizer.timestamp_begin)
      single_timestamp_ending = timestamp_tokens[-2:].tolist() == [False, True]

      consecutive = torch.where(timestamp_tokens[:-1] & timestamp_tokens[1:])[0]
      consecutive.add_(1)
      if len(consecutive) > 0:
          # if the output contains two consecutive timestamp tokens
          slices = consecutive.tolist()
          if single_timestamp_ending:
              slices.append(len(tokens))

          last_slice = 0
          for current_slice in slices:
              sliced_tokens = tokens[last_slice:current_slice]
              start_timestamp_pos = (
                  sliced_tokens[0].item() - tokenizer.timestamp_begin
              )
              end_timestamp_pos = (
                  sliced_tokens[-1].item() - tokenizer.timestamp_begin
              )
              current_segments.append(
                  new_segment(
                      start=time_offset + start_timestamp_pos * time_precision,
                      end=time_offset + end_timestamp_pos * time_precision,
                      tokens=sliced_tokens,
                      result=result,
                  )
              )
              last_slice = current_slice

          if single_timestamp_ending:
              # single timestamp at the end means no speech after the last timestamp.
              seek += segment_size
          else:
              # otherwise, ignore the unfinished segment and seek to the last timestamp
              last_timestamp_pos = (
                  tokens[last_slice - 1].item() - tokenizer.timestamp_begin
              )
              seek += last_timestamp_pos * input_stride
      else:
          duration = segment_duration
          timestamps = tokens[timestamp_tokens.nonzero().flatten()]
          if (
              len(timestamps) > 0
              and timestamps[-1].item() != tokenizer.timestamp_begin
          ):
              # no consecutive timestamps but it has a timestamp; use the last one.
              last_timestamp_pos = (
                  timestamps[-1].item() - tokenizer.timestamp_begin
              )
              duration = last_timestamp_pos * time_precision

          current_segments.append(
              new_segment(
                  start=time_offset,
                  end=time_offset + duration,
                  tokens=tokens,
                  result=result,
              )
          )
          seek += segment_size

      if result.temperature > 0.5:
          # do not feed the prompt tokens if a high temperature was used
          prompt_reset_since = len(all_tokens)

      # if a segment is instantaneous or does not contain text, clear it
      for i, segment in enumerate(current_segments):
          if segment["start"] == segment["end"] or segment["text"].strip() == "":
              segment["text"] = ""
              segment["tokens"] = []
              segment["words"] = []

      all_segments.extend(
          [
              {"id": i, **segment}
              for i, segment in enumerate(
                  current_segments, start=len(all_segments)
              )
          ]
      )
      all_tokens.extend(
          [token for segment in current_segments for token in segment["tokens"]]
      )


  return tokenizer.decode(all_tokens)

In [18]:
text_transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    jiwer.Strip(),
    jiwer.ReduceToListOfListOfWords()
]) 

def wer(target, output):
  return jiwer.wer(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)
  
def wil(target, output):
  return jiwer.wil(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)


In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, cfx, p, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

# Baseline

## Fetching the Dataset

In [4]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"UserName","key":"Key"}') 
#     # Put your kaggle username & key here
with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"sma2023","key":"6c819f763f537a6b8bbb60cb11520dbf"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=2761abf8cabcf8d60f78714a24ae0b4b7d40604aa99ffa664c0530e61002775e
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [5]:
!kaggle datasets download -d rtatman/speech-accent-archive
!unzip -qo 'speech-accent-archive.zip' -d '/content/data'

 99% 860M/865M [00:08<00:00, 65.5MB/s]
100% 865M/865M [00:08<00:00, 106MB/s] 


## Define the two metrics

In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00


In [ ]:
import jiwer

In [ ]:
jiwer.wer("the cat", "cat the")

1.0

In [ ]:
f = open("/content/data/reading-passage.txt")
target = ""
for line in f:
  target += line

text_transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    # jiwer.ReduceToSingleSentence(),
    jiwer.Strip(),
    jiwer.SubstituteRegexes({r"6": r"six", r"5": r"five", r"3": r"three"}),
    jiwer.ReduceToListOfListOfWords()
]) 

def wer(output):
  return jiwer.wer(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)
  
def wil(output):
  return jiwer.wil(
    target, 
    output, 
    truth_transform=text_transformation, 
    hypothesis_transform=text_transformation)


## Transcribe and recording the data

In [ ]:
# open the speaker_all.csv
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
speakers = pd.read_csv('/content/data/speakers_all.csv')
speakers['wer'] = 1.0
speakers['wil'] = 1.0
cnt = 0
batch_bar   = tqdm(total=len(speakers), dynamic_ncols=True, leave=False, position=0)
for index, row in speakers.iterrows():
  if row['file_missing?']==False:
    file_name = row['filename']
    transcription = whisper.transcribe(model = model, audio = '/content/data/recordings/recordings/'+file_name+'.mp3', fp16=False)['text']
    speakers.at[index,'wer'] = wer(transcription)
    speakers.at[index,'wil'] = wil(transcription)
  batch_bar.update()
batch_bar.close()
speakers.to_csv('results.csv')


  0%|          | 0/2172 [00:00<?, ?it/s]